# Reprise du code de SEBA


In [5]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import pyproj
import os


In [6]:
# Liste de tes 4 fichiers GeoTIFF
geotiff_files = [
    'data_input/data_processed/ghs_gaza_pop_added_full_extent.tif',
    'data_input/data_processed/ghs_gaza_strip.tif', 
    'data_input/data_processed/ghs_geneva.tif',
    'data_input/data_processed/ghs_schaffhausen.tif'
]

In [7]:
# Créer le dossier de sortie s'il n'existe pas
os.makedirs('data_output/visualisation_3d', exist_ok=True)

print("=== DIAGNOSTIC COMPLET DES GEOTIFF ===\n")

for i, file in enumerate(geotiff_files):
    print(f"Fichier {i+1}: {file}")
    print("-" * 50)
    
    try:
        with rasterio.open(file) as src:
            # Métadonnées de base
            print(f"Dimensions: {src.width} x {src.height} pixels")
            print(f"Bandes: {src.count}")
            print(f"CRS: {src.crs}")
            print(f"Bounds: {src.bounds}")
            
            # Lire les données
            data = src.read(1)
            print(f"Type de données: {data.dtype}")
            print(f"Forme du array: {data.shape}")
            
            # Statistiques brutes
            print(f"Valeurs uniques (échantillon): {np.unique(data.flatten())[:10]}")
            print(f"Min brut: {np.min(data)}")
            print(f"Max brut: {np.max(data)}")
            print(f"Moyenne brute: {np.mean(data):.2f}")
            
            # Compter les valeurs spéciales
            n_nan = np.sum(np.isnan(data))
            n_neg = np.sum(data < 0)
            n_zero = np.sum(data == 0)
            n_pos = np.sum(data > 0)
            
            print(f"NaN: {n_nan}")
            print(f"Négatifs: {n_neg}")
            print(f"Zéros: {n_zero}")
            print(f"Positifs: {n_pos}")
            
            # Nettoyer les données (supprimer les négatifs)
            data_clean = np.where(data < 0, np.nan, data)
            data_clean = np.where(data_clean == 0, np.nan, data_clean)  # Optionnel : supprimer aussi les zéros
            
            n_valid = np.sum(~np.isnan(data_clean))
            print(f"Pixels valides après nettoyage: {n_valid}")
            
            if n_valid > 0:
                print(f"Min nettoyé: {np.nanmin(data_clean):.2f}")
                print(f"Max nettoyé: {np.nanmax(data_clean):.2f}")
                print(f"Moyenne nettoyée: {np.nanmean(data_clean):.2f}")
                print(f"Médiane nettoyée: {np.nanmedian(data_clean):.2f}")
                
                # Percentiles pour comprendre la distribution
                p90 = np.nanpercentile(data_clean, 90)
                p95 = np.nanpercentile(data_clean, 95)
                p99 = np.nanpercentile(data_clean, 99)
                print(f"90e percentile: {p90:.2f}")
                print(f"95e percentile: {p95:.2f}")
                print(f"99e percentile: {p99:.2f}")
            else:
                print("AUCUNE DONNÉE VALIDE !")
            
    except Exception as e:
        print(f"ERREUR lors de la lecture: {e}")
    
    print("\n")

print("=== DIAGNOSTIC TERMINÉ ===")
print("Si tu vois des données valides, on peut passer à l'étape suivante !")

=== DIAGNOSTIC COMPLET DES GEOTIFF ===

Fichier 1: data_input/data_processed/ghs_gaza_pop_added_full_extent.tif
--------------------------------------------------
Dimensions: 419 x 450 pixels
Bandes: 1
CRS: EPSG:4326
Bounds: BoundingBox(left=34.21874988686396, bottom=31.219583410589177, right=34.56791655214766, top=31.594583409079952)
Type de données: float32
Forme du array: (450, 419)
Valeurs uniques (échantillon): [  0.      172.71988 172.72037 172.72041 172.72069 172.72086 172.72093
 172.72113 172.72118 172.7213 ]
Min brut: nan
Max brut: nan
Moyenne brute: nan
NaN: 137796
Négatifs: 0
Zéros: 41845
Positifs: 8909
Pixels valides après nettoyage: 8909
Min nettoyé: 172.72
Max nettoyé: 632.09
Moyenne nettoyée: 234.94
Médiane nettoyée: 187.21
90e percentile: 370.55
95e percentile: 401.24
99e percentile: 452.09


Fichier 2: data_input/data_processed/ghs_gaza_strip.tif
--------------------------------------------------
Dimensions: 419 x 450 pixels
Bandes: 1
CRS: EPSG:4326
Bounds: BoundingBox

In [9]:
# Create custom colormap with brighter colors
custom_colors_darkmode = [
    (0.0, '#575247'),     # Light beige (start at 0)
    (0.1, '#691A1F'),     # Medium green (middle)
    (1.0, '#E78D38')      # Dark green (end at 1)
]
custom_colors = [
    (0.0, '#DCD8C9'),     # Light beige (start at 0)
    (0.1, '#95D0CF'),     # Medium green (middle)
    (1.0, '#A757A3')      # Dark green (end at 1)
]

# First pass: find global min and max values and pixel sizes
all_data = []
pixel_sizes = []
for file in geotiff_files:
    with rasterio.open(file) as src:
        data = src.read(1)
        # Garder les zéros, supprimer seulement les négatifs
        data = np.where(data < 0, np.nan, data)
        all_data.append(data)
        transform = src.transform
        pixel_size = (abs(transform[0]), abs(transform[4]))
        pixel_sizes.append(pixel_size)

vmin = min(np.nanmin(data) for data in all_data)
vmax = max(np.nanmax(data) for data in all_data)

print(f"Échelle globale: min={vmin:.2f}, max={vmax:.2f}")

# Find the largest dimensions to maintain scale
max_pixel_size = max(max(sizes) for sizes in pixel_sizes)
scale_factors = [(max_pixel_size/px[0], max_pixel_size/px[1]) for px in pixel_sizes]

print("Facteurs d'échelle calculés:")
for i, (file, scale) in enumerate(zip(geotiff_files, scale_factors)):
    print(f"  {file.split('/')[-1]}: {scale[0]:.3f} x {scale[1]:.3f}")

# Create individual plots
for idx, (file, data, scale) in enumerate(zip(geotiff_files, all_data, scale_factors)):
    print(f"\nProcessing {file}")
    
    # Create both light and dark mode versions
    for mode in ['light', 'dark']:
        print(f"Creating {mode} mode version...")
        
        # Select appropriate colormap based on mode
        colors = custom_colors_darkmode if mode == 'dark' else custom_colors
        custom_cmap = LinearSegmentedColormap.from_list('custom', colors)
        
        # Create figure with transparent background
        fig = plt.figure(figsize=(12, 10), dpi=300)
        ax = fig.add_subplot(111, projection='3d')
        
        # Make both figure and axis background transparent
        fig.patch.set_alpha(0.0)
        ax.patch.set_alpha(0.0)
        
        # Set axis pane colors to transparent
        ax.xaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
        ax.yaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
        ax.zaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
        
        # Downsample the data
        downsample_factor = 2  # Ajuste selon la performance souhaitée
        data_downsampled = data[::downsample_factor, ::downsample_factor]
        
        # Create coordinates for bars
        y, x = np.meshgrid(np.arange(data_downsampled.shape[1]), 
                          np.arange(data_downsampled.shape[0]))
        x = x.flatten() * downsample_factor * scale[0]
        y = y.flatten() * downsample_factor * scale[1]
        z = np.zeros_like(x)
        dx = dy = downsample_factor * max(scale) * 0.95
        dz = data_downsampled.flatten()
        
        # Remove only NaN values (keep zeros)
        mask = ~np.isnan(dz)
        x, y, z, dz = x[mask], y[mask], z[mask], dz[mask]
        
        print(f"  - {len(dz)} barres à dessiner (dont {np.sum(dz == 0)} zéros)")
        
        # Create the bar plot
        bars = ax.bar3d(x, y, z, dx, dy, dz, 
                        zsort='max',
                        shade=True)

        # Get the colors for all faces
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        face_colors = custom_cmap(norm(dz))
        
        # Modify colors for each face (6 faces per bar)
        facecolors = np.zeros((len(dz) * 6, 4))
        for i in range(len(dz)):
            color = face_colors[i]
            dark_color = color * [0.5, 0.5, 0.5, 1.0]  # Face du dessous plus sombre
            facecolors[i*6:(i+1)*6] = color
            facecolors[i*6 + 2] = dark_color  # Face du dessous

        # Apply the modified colors
        bars.set_facecolors(facecolors)

        # Set boundaries with proper scaling
        if len(x) > 0:
            x_min, x_max = np.min(x), np.max(x) + dx
            y_min, y_max = np.min(y), np.max(y) + dy
            x_padding = (x_max - x_min) * 0.02
            y_padding = (y_max - y_min) * 0.02
            
            # Set axis limits
            ax.set_xlim(x_min - x_padding, x_max + x_padding)
            ax.set_ylim(y_min - y_padding, y_max + y_padding)
            ax.set_zlim(vmin, vmax * 0.8)  # Un peu moins haut pour mieux voir
        
        ax.set_axis_off()
        ax.view_init(elev=30, azim=0)

        # Calculate area
        with rasterio.open(file) as src:
            pixel_size_x = abs(src.transform[0])
            pixel_size_y = abs(src.transform[4])
            pixel_area = pixel_size_x * pixel_size_y
            valid_pixels = np.sum(~np.isnan(data))
            total_area_sq_km = (valid_pixels * pixel_area) / 1_000_000

        # Add title with clean name
        clean_name = file.split('/')[-1].replace('ghs_', '').replace('.tif', '')
        if 'gaza_pop_added_full_extent' in clean_name:
            clean_name = 'Gaza (étendu)'
        elif 'gaza_strip' in clean_name:
            clean_name = 'Gaza (bande)'
        elif 'geneva' in clean_name:
            clean_name = 'Genève'
        elif 'schaffhausen' in clean_name:
            clean_name = 'Schaffhouse'
            
        title = f"{clean_name}\nSurface: {total_area_sq_km:.1f} km²"
        ax.set_title(title, pad=20, fontsize=14)
        
        # Add colorbar
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label('Densité de population\n(hab./km²)', fontsize=12)
        
        # Adjust layout
        plt.subplots_adjust(left=0.02, right=0.9)
        
        # Save with appropriate suffix
        suffix = '_dm' if mode == 'dark' else ''
        output_filename = f"data_output/visualisation_3d/{clean_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}{suffix}.png"
        plt.savefig(output_filename, 
                    dpi=300, 
                    bbox_inches='tight',
                    pad_inches=0.1,
                    transparent=True,
                    facecolor='none')
        print(f"  Saved: {output_filename}")
        
        plt.close()

print(f"\n=== TERMINÉ ===")
print(f"Toutes les visualisations sauvées dans: data_output/visualisation_3d/")
print(f"Échelle conservée entre toutes les visualisations")
print(f"Modes disponibles: normal et dark mode (_dm)")

Échelle globale: min=0.00, max=632.09
Facteurs d'échelle calculés:
  ghs_gaza_pop_added_full_extent.tif: 1.000 x 1.000
  ghs_gaza_strip.tif: 1.000 x 1.000
  ghs_geneva.tif: 1.000 x 1.000
  ghs_schaffhausen.tif: 1.000 x 1.000

Processing data_input/data_processed/ghs_gaza_pop_added_full_extent.tif
Creating light mode version...
  - 12696 barres à dessiner (dont 10468 zéros)
  Saved: data_output/visualisation_3d/gaza_étendu.png
Creating dark mode version...
  - 12696 barres à dessiner (dont 10468 zéros)
  Saved: data_output/visualisation_3d/gaza_étendu_dm.png

Processing data_input/data_processed/ghs_gaza_strip.tif
Creating light mode version...
  - 12696 barres à dessiner (dont 2242 zéros)
  Saved: data_output/visualisation_3d/gaza_bande.png
Creating dark mode version...
  - 12696 barres à dessiner (dont 2242 zéros)
  Saved: data_output/visualisation_3d/gaza_bande_dm.png

Processing data_input/data_processed/ghs_geneva.tif
Creating light mode version...
  - 12122 barres à dessiner (dont